In [ ]:
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [ ]:
# ----------------------------- IO helpers --------------------------------- #

def read_xyz(path: Path):
    data=[]
    with path.open('r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            s=line.strip()
            if not s: continue
            try:
                vals=[float(v) for v in s.split()]
            except ValueError:
                continue
            if len(vals)>=3:
                data.append(vals)
    arr=np.asarray(data,float)
    xyz=arr[:,:3]
    rgb=None
    if arr.shape[1]>=6:
        rgb=np.clip(arr[:,3:6],0,255)
    return xyz, rgb

def read_labelled_xyz(path: Path):
    data=[]
    with path.open('r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            s=line.strip()
            if not s: continue
            try:
                vals=[float(v) for v in s.split()]
            except ValueError:
                continue
            if len(vals)>=4:
                data.append(vals)
    arr=np.asarray(data,float)
    return arr[:,:3], arr[:,-1].astype(int)

def read_obj_lines(path: Path):
    verts=[]; segs=[]
    with path.open('r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            s=line.strip()
            if not s or s.startswith('#'): continue
            if s.startswith('v '):
                _,x,y,z,*_ = s.split(); verts.append([float(x),float(y),float(z)])
            elif s.startswith('l '):
                ids=[int(tok.split('/')[0]) for tok in s.split()[1:]]
                for a,b in zip(ids,ids[1:]): segs.append((a-1,b-1))
    return np.asarray(verts,float), segs

# --------------------------- Sampling ------------------------------------- #

def random_subsample(arr: np.ndarray, k: int, seed: int=0):
    if len(arr)<=k: return arr
    rng=np.random.default_rng(seed)
    return arr[rng.choice(len(arr),k,replace=False)]

# --------------------------- Color helpers -------------------------------- #

def face_colors(fid: np.ndarray):
    palette = (
        ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']+
        ['#393b79','#637939','#8c6d31','#843c39','#7b4173','#3182bd','#e6550d','#31a354','#756bb1','#636363']
    )
    colors=np.empty(len(fid),dtype=object)
    neg=fid<0; colors[neg]='rgba(160,160,160,0.3)'
    pos=~neg
    if np.any(pos):
        idx=fid[pos]%len(palette)
        colors[pos]=np.array(palette)[idx]
    return colors

# ------------------------------ Plotting ---------------------------------- #

def make_subplots_figure(xyz_before, rgb_before, xyz_after, fid, V, segs, point_size, line_width):
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'scene'},{'type':'scene'}]], subplot_titles=('Before (RGB)','After (By FACE_ID)'))

    # BEFORE layer with RGB coloring
    if rgb_before is not None:
        col=['rgb(%d,%d,%d)'%(int(r),int(g),int(b)) for r,g,b in rgb_before]
    else:
        col='rgba(100,100,100,1.0)'
    fig.add_trace(go.Scatter3d(x=xyz_before[:,0],y=xyz_before[:,1],z=xyz_before[:,2],mode='markers',
        marker=dict(size=point_size,color=col,opacity=0.8),name='Before (RGB)'),row=1,col=1)

    # AFTER layer colored by FACE_ID
    colors=face_colors(fid)
    fig.add_trace(go.Scatter3d(x=xyz_after[:,0],y=xyz_after[:,1],z=xyz_after[:,2],mode='markers',
        marker=dict(size=point_size,color=colors,opacity=0.9),name='After (FACE_ID)'),row=1,col=2)

    # Wireframe overlay
    if len(V)>0 and len(segs)>0:
        x=[];y=[];z=[]
        for a,b in segs:
            pa,pb=V[a],V[b]
            x+=[pa[0],pb[0],None]
            y+=[pa[1],pb[1],None]
            z+=[pa[2],pb[2],None]
        wire=go.Scatter3d(x=x,y=y,z=z,mode='lines',line=dict(width=line_width,color='rgba(30,144,255,0.8)'),name='Wireframe')
        fig.add_trace(wire,row=1,col=1)
        fig.add_trace(wire,row=1,col=2)

        fig.update_layout(scene=dict(aspectmode='data'),scene2=dict(aspectmode='data'),margin=dict(l=0,r=0,t=40,b=0))
    #     fig.update_layout(
    #     scene=dict(
    #         domain=dict(x=[0.0, 0.48], y=[0, 1]),
    #         aspectmode='data'
    #     ),
    #     scene2=dict(
    #         domain=dict(x=[0.52, 1.0], y=[0, 1]),
    #         aspectmode='data'
    #     ),
    #     margin=dict(l=0, r=0, t=40, b=0)
    # )
        # fig.update_layout(width=1600, height=800)
    # fig.update_layout(scene=dict(aspectmode='cube'), scene2=dict(aspectmode='cube'))
    return fig


In [ ]:
# ------------------------------ Main Test --------------------------------- #
# Manual configuration — replace paths/values as needed
xyz_file = Path("data/Entry-level/train/xyz/2.xyz")
xyz_labelled_file = Path("outputs/2.xyz")
obj_file = Path("data/Entry-level/train/wireframe/2.obj")

SAMPLE = 200000
POINT_SIZE = 2.0
LINE_WIDTH = 2.0
OUT_HTML = None  # or Path("/path/to/out.html")

def test_main():
    xyz0, rgb0 = read_xyz(xyz_file)
    # display(rgb0)
    xyz1, fid = read_labelled_xyz(xyz_labelled_file)
    V, segs = read_obj_lines(obj_file)

    if SAMPLE and SAMPLE > 0:
        xyz0 = random_subsample(xyz0, SAMPLE, 0)
        if rgb0 is not None:
            rgb0 = rgb0[: len(xyz0)]
        xyz1 = random_subsample(xyz1, SAMPLE, 1)
        fid = fid[: len(xyz1)]

    fig = make_subplots_figure(xyz0, rgb0, xyz1, fid, V, segs, POINT_SIZE, LINE_WIDTH)

    if OUT_HTML:
        pio.write_html(fig, file=str(OUT_HTML), auto_open=False, include_plotlyjs='cdn')
        print(f"Saved interactive HTML to {OUT_HTML}")

    fig.show()

if __name__ == "__main__":
    test_main()